In [2]:
import torch
import random
import pandas as pd
import json
import numpy as np
import torch.backends.cudnn as cudnn

In [3]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)
# seed 결과가달라짐 3~4%

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print('device:', device)

device: cuda:0


In [4]:
print('-'*10)
print('Data Loading Start!!')
print('-'*10)

## dataset class
path = "/data/ephemeral/home/data/aug_data_x10.jsonl"

with open(path) as f:
    data = [json.loads(line) for line in f]
data = pd.DataFrame(data)
print('origin_data:', data.shape)


----------
Data Loading Start!!
----------
origin_data: (42720, 5)


In [5]:
data.columns

Index(['docid', 'question', 'content', 'src', 'new_domains'], dtype='object')

In [6]:
data['new_domains'].unique()

array(['human_aging', 'medical_genetics', 'high_school_biology',
       'college_chemistry', 'college_physics', 'conceptual_physics',
       'global_facts', 'None', 'unknown', 'computer_security',
       'high_school_chemistry', 'anatomy', 'nutrition', 'human_sexuality',
       'astronomy', 'high_school_computer_science', 'virology',
       'electrical_engineering', 'college_medicine', 'college_biology',
       'college_computer_science', 'human_aging, nutrition',
       'high_school_physics', 'geology', 'art', 'college_science',
       'safety', 'civil_engineering', 'new_technology_in_industry',
       'engineering', 'data_visualization',
       'human_sexuality, medical_genetics', 'environmental_science',
       'music_performance', 'logistics', 'None, medical_genetics',
       'astronomy, college_physics', 'construction_tools',
       'astronomy, college_biology', 'astronomy, conceptual_physics',
       'anatomy, college_chemistry', 'safety_education',
       'investigation_records'

In [5]:
data2 = data[~data['new_domains'].isin(['None','unknown'])]

In [6]:
data2.shape

(41580, 5)

In [18]:
data2['new_domains'].unique()

array(['human_aging', 'medical_genetics', 'high_school_biology',
       'college_chemistry', 'college_physics', 'conceptual_physics',
       'global_facts', 'computer_security', 'high_school_chemistry',
       'anatomy', 'nutrition', 'human_sexuality', 'astronomy',
       'high_school_computer_science', 'virology',
       'electrical_engineering', 'college_medicine', 'college_biology',
       'college_computer_science', 'human_aging, nutrition',
       'high_school_physics', 'geology', 'art', 'college_science',
       'safety', 'civil_engineering', 'new_technology_in_industry',
       'engineering', 'data_visualization',
       'human_sexuality, medical_genetics', 'environmental_science',
       'music_performance', 'logistics', 'None, medical_genetics',
       'astronomy, college_physics', 'construction_tools',
       'astronomy, college_biology', 'astronomy, conceptual_physics',
       'anatomy, college_chemistry', 'safety_education',
       'investigation_records'], dtype=object)

In [127]:
data2[data2['new_domains']=='astronomy, college_biology']

,docid,question,content,src,new_domains
33360,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들은 어떤 대상을 연구하나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33361,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들은 어떤 객체를 연구하나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33362,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자의 연구 방법에는 어떤 공통점이 있나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33363,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치의 예시로 어떤 것들이 있나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33364,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들이 사용하는 장치는 무엇인가요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33365,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들이 사용하는 장치는 무엇인가요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33366,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치를 사용하면 어떤 이점이 있나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33367,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자는 어떤 분야의 연구자들인가요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33368,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자가 연구하는 대상은 어떻게 다르나요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 천체를 관찰하고, 생물학자들은 극도로 작은 객체를 연구합니다. 이 두 분야의 연구자들은 다른 대상을 다루지만, 그들의 연구 방법에는 공통점이 있습니다. 천문학자와 생물학자들은 모두 광학 장치를 사용하여 발견을 합니다. 광학 장치는 망원경, 현미경 등 다양한 형태로 사용되며, 이를 통해 더 나은 관찰과 연구가 가능해집니다. 따라서 천문학자와 생물학자들은 공통적으로 광학 장치를 활용하여 자신들의 분야에서 발견을 이루고 있습니다.",ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33369,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치가 발견에 미치는 영향은 무엇인가요?,"천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우 멀리 떨어진 

In [7]:
data2_expanded = data2.copy()
data2_expanded['new_domains'] = data2_expanded['new_domains'].str.split(', ')
data2_expanded = data2_expanded.explode('new_domains').reset_index(drop=True)


In [20]:
data2_expanded[data2_expanded['docid']=='c73343e8-395d-40d0-854a-529d11c4e194']

,docid,question,content,src,new_domains
32520,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들은 어떤 대상을 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32521,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들은 어떤 대상을 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology
32522,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들은 어떤 객체를 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32523,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들은 어떤 객체를 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology
32524,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자의 연구 방법에는 어떤 공통점이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32525,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자의 연구 방법에는 어떤 공통점이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology
32526,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치의 예시로 어떤 것들이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32527,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치의 예시로 어떤 것들이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology
32528,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들이 사용하는 장치는 무엇인가요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,astronomy
32529,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들이 사용하는 장치는 무엇인가요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,college_biology


In [8]:
data2_expanded.shape

(41650, 5)

In [13]:
data2_expanded.new_domains.unique()

array(['human_aging', 'medical_genetics', 'high_school_biology',
       'college_chemistry', 'college_physics', 'conceptual_physics',
       'global_facts', 'computer_security', 'high_school_chemistry',
       'anatomy', 'nutrition', 'human_sexuality', 'astronomy',
       'high_school_computer_science', 'virology',
       'electrical_engineering', 'college_medicine', 'college_biology',
       'college_computer_science', 'high_school_physics', 'geology',
       'art', 'college_science', 'safety', 'civil_engineering',
       'new_technology_in_industry', 'engineering', 'data_visualization',
       'environmental_science', 'music_performance', 'logistics', 'None',
       'construction_tools', 'safety_education', 'investigation_records'],
      dtype=object)

In [9]:
data2 = data2_expanded[~data2_expanded['new_domains'].isin(['None'])]

In [10]:
len(data2)

41640

In [11]:
import pandas as pd
aug10 = pd.read_json('/data/ephemeral/home/sample_submission_aug10.csv', lines=True)

In [69]:
aug10['topk']

0      [c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d, e227a02...
1      [79c93deb-fe60-4c81-8d51-cb7400a0a156, 4f0ea39...
2      [25de4ffd-cee4-4f27-907e-fd6b802c6ede, 0c2d526...
3      [bd91bda8-351e-4683-bb1a-8254f93e2376, 5d48fa4...
4      [38686456-b993-4cbb-af0d-1c53df2f3e12, e532908...
                             ...                        
215    [fed07bdf-90e8-4236-bc5e-43eb5fe17061, 2452823...
216    [a662feb5-e069-44d8-b148-4641039d2329, e216195...
217    [dc398ded-03b2-4113-8ae6-9161cb5a545c, 5973b08...
218    [466f5811-9b37-4589-9571-bafadd92c349, 6c21a0d...
219    [bf023a9d-47f3-4d77-8bf1-5a854b5403a7, 25eee08...
Name: topk, Length: 220, dtype: object

In [12]:
import pandas as pd

# 각 row의 리스트에서 앞의 5개 항목만 가져오기
aug10['topk'] = aug10['topk'].apply(lambda x: x[:10] if isinstance(x, list) else x)

# Merging the lists in the 'topk' column into a single string
aug10['topk_combined'] = aug10['topk'].apply(lambda x: ', '.join(x))

combined_list = [item.strip() for sublist in aug10['topk_combined'].str.split(',') for item in sublist]

print(len(combined_list))

# Remove empty strings and strip any trailing commas from the combined list
cleaned_combined_list = set([item.strip() for item in combined_list if item.strip()])

print(len(cleaned_combined_list))



1999
1545


In [13]:
data3 = data2[data2['docid'].isin(cleaned_combined_list)]

In [14]:
len(data3)

15140

In [17]:
import torch
from sentence_transformers import SentenceTransformer, util, InputExample
from tqdm import tqdm
from itertools import combinations 

# Sentence-BERT 모델을 사용해 질문 임베딩 생성
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU가 있으면 'cuda', 없으면 'cpu'
model = SentenceTransformer('jhgan/ko-sroberta-multitask').to(device)  # 모델을 GPU로 전송

def create_labelled_pairs_with_similarity(data, threshold=0.7):
    # 데이터를 도메인별로 그룹화
    grouped_by_domain = data.groupby('new_domains')
    positive_pairs = []
    negative_pairs = []

    # 도메인별로 긍정 쌍 생성
    for domain, group in tqdm(grouped_by_domain, desc="Processing domains"):
        grouped_by_doc = group.groupby('docid')
        
        # 동일 문서 내의 질문들로 긍정적인 쌍 생성
        for docid, questions_in_doc in grouped_by_doc:
            questions = list(questions_in_doc['question'])
            for q1, q2 in combinations(questions, 2):
                positive_pairs.append(InputExample(texts=[q1, q2], label=1))
        
        # 도메인 내 다른 문서 간 부정적인 쌍 생성 (유사도 기반 필터링)
        docids = list(grouped_by_doc.groups.keys())
        for i in range(len(docids)):
            for j in range(i + 1, len(docids)):
                questions1 = list(grouped_by_doc.get_group(docids[i])['question'])
                questions2 = list(grouped_by_doc.get_group(docids[j])['question'])
                
                # 질문 임베딩 생성 (GPU에서 수행)
                embeddings1 = model.encode(questions1, convert_to_tensor=True, device=device)
                embeddings2 = model.encode(questions2, convert_to_tensor=True, device=device)

                # 유사도 계산 (임계값보다 높은 쌍만 부정 쌍으로 추가)
                cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
                for idx1, q1 in enumerate(questions1):
                    for idx2, q2 in enumerate(questions2):
                        if cosine_scores[idx1, idx2] > threshold:  # 유사도가 일정 임계값 이상일 때만
                            negative_pairs.append(InputExample(texts=[q1, q2], label=0))

    return positive_pairs + negative_pairs

# 데이터 생성
training_pairs = create_labelled_pairs_with_similarity(data3, threshold=0.7)  # 유사도 임계값을 0.7로 설정

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing domains: 100%|██████████| 26/26 [24:18<00:00, 56.11s/it] 


In [18]:
len(training_pairs)

138102

In [21]:
import pickle

# training_pairs 저장
with open('training_pairs.pkl', 'wb') as f:
    pickle.dump(training_pairs, f)


In [25]:
import pickle

# training_pairs 불러오기
with open('training_pairs.pkl', 'rb') as f:
    training_pairs = pickle.load(f)

# 이제 loaded_training_pairs를 사용할 수 있습니다.

In [26]:
# Print the first 10 pairs to see their content and labels
for example in training_pairs[:30]:
    print("Query:", example.texts[0])
    print("Document:", example.texts[1])
    print("Label:", example.label)
    print("-" * 30)


Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 기관계는 어떤 기능을 조절하나요?
Label: 1
------------------------------
Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 뇌는 어떤 신호를 생성하고 전달하나요?
Label: 1
------------------------------
Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 척수의 주요 기능은 무엇인가요?
Label: 1
------------------------------
Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 신경은 어떻게 신호를 전달하나요?
Label: 1
------------------------------
Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 뇌와 척수는 어떤 관계가 있나요?
Label: 1
------------------------------
Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 기관계는 왜 중요한가요?
Label: 1
------------------------------
Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 아그네스는 무엇을 배웠나요?
Label: 1
------------------------------
Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 신경은 신체의 어떤 부분과 연결되나요?
Label: 1
------------------------------
Query: 뇌, 척수, 신경의 역할은 무엇인가요?
Document: 뇌, 척수, 신경이 함께 협력하는 이유는 무엇인가요?
Label: 1
------------------------------
Query: 기관계는 어떤 기능을 조절하나요?
Document: 뇌는 어떤 신호를 생성하고 전달하나요?
Label: 1
------------------------------
Query: 기관계는 

In [24]:
for example in training_pairs[30:50]:
    print("Query:", example.texts[0])
    print("Document:", example.texts[1])
    print("Label:", example.label)
    print("-" * 30)

Query: 신경은 어떻게 신호를 전달하나요?
Document: 뇌와 척수는 어떤 관계가 있나요?
Label: 1
------------------------------
Query: 신경은 어떻게 신호를 전달하나요?
Document: 기관계는 왜 중요한가요?
Label: 1
------------------------------
Query: 신경은 어떻게 신호를 전달하나요?
Document: 아그네스는 무엇을 배웠나요?
Label: 1
------------------------------
Query: 신경은 어떻게 신호를 전달하나요?
Document: 신경은 신체의 어떤 부분과 연결되나요?
Label: 1
------------------------------
Query: 신경은 어떻게 신호를 전달하나요?
Document: 뇌, 척수, 신경이 함께 협력하는 이유는 무엇인가요?
Label: 1
------------------------------
Query: 뇌와 척수는 어떤 관계가 있나요?
Document: 기관계는 왜 중요한가요?
Label: 1
------------------------------
Query: 뇌와 척수는 어떤 관계가 있나요?
Document: 아그네스는 무엇을 배웠나요?
Label: 1
------------------------------
Query: 뇌와 척수는 어떤 관계가 있나요?
Document: 신경은 신체의 어떤 부분과 연결되나요?
Label: 1
------------------------------
Query: 뇌와 척수는 어떤 관계가 있나요?
Document: 뇌, 척수, 신경이 함께 협력하는 이유는 무엇인가요?
Label: 1
------------------------------
Query: 기관계는 왜 중요한가요?
Document: 아그네스는 무엇을 배웠나요?
Label: 1
------------------------------
Query: 기관계는 왜 중요한가요?
Document: 신경은 신체의 어떤 부분과 

In [ ]:
# https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/cross-encoder/training_stsbenchmark.py

In [28]:
import logging
import math
from datetime import datetime
import torch
from torch.utils.data import DataLoader
from torch.nn import BCEWithLogitsLoss
from sentence_transformers import LoggingHandler
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # Import tqdm for the progress bar

# Setup logging
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()])

# Define model parameters
model_name = "jhgan/ko-sroberta-multitask"
#model_name = "/data/ephemeral/home/code/sroberta_finetuned_epoch_1"
train_batch_size = 32
num_epochs = 10
model_save_path = f"output/training_{model_name.replace('/', '-')}_binary_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

# Initialize the model and tokenizer
model = CrossEncoder(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Split the data into train, validation, and test sets
train_val_pairs, test_pairs = train_test_split(training_pairs, test_size=0.2, random_state=42)
train_pairs, validation_pairs = train_test_split(train_val_pairs, test_size=0.2, random_state=42)

# Custom collate function to tokenize the data
def collate_fn(batch):
    texts = [pair.texts for pair in batch]
    labels = [pair.label for pair in batch]
    
    # Tokenize the texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    
    # Convert labels to tensors
    labels = torch.tensor(labels, dtype=torch.float)
    
    return inputs, labels

# Create DataLoader for train, validation, and test sets with custom collate function
train_dataloader = DataLoader(train_pairs, shuffle=True, batch_size=train_batch_size, collate_fn=collate_fn)
validation_dataloader = DataLoader(validation_pairs, shuffle=False, batch_size=train_batch_size, collate_fn=collate_fn)
test_dataloader = DataLoader(test_pairs, shuffle=False, batch_size=train_batch_size, collate_fn=collate_fn)

# Prepare evaluator with validation set
dev_samples = [pair for pair in validation_pairs]
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name="aug-dev")
# Correlation 값은 모델이 예측한 값과 실제 레이블 사이의 상관관계
# Pearson 값이 0.9021인 경우, 모델이 예측한 값과 실제 레이블 간에 강한 양의 선형 관계가 있다는 것을 의미
# Spearman 값이 0.8399인 경우, 모델이 예측한 값과 실제 값의 순위 간에 높은 일관성이 있다는 것을 의미

# Configure warmup steps (10% of training steps)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

# Loss function: BCEWithLogitsLoss (which includes sigmoid activation)
loss_fn = BCEWithLogitsLoss()

# Train the model using the fit method with tqdm progress bar
logging.info("Starting training...")
import torch.optim as optim

# Define optimizer
optimizer = optim.AdamW(model.model.parameters(), lr=2e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check for available device
model.model.to(device)  # Move model to GPU if available

for epoch in range(num_epochs):
    model.model.train()  # Ensure the model is in training mode
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", leave=False)

    for step, (inputs, labels) in enumerate(progress_bar):
        # Move inputs and labels to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}  # Ensure all inputs are on the correct device
        labels = labels.to(device)  # Ensure labels are on the correct device

        # Manually compute outputs and update progress bar
        outputs = model.model(**inputs).logits
        loss = torch.nn.BCEWithLogitsLoss()(outputs.view(-1), labels)

        # Zero gradients, backward pass, and optimizer step
        optimizer.zero_grad()  # Manually zero the gradients
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimizer step to update weights

        # Update the progress bar with the current loss
        progress_bar.set_postfix({'loss': loss.item()})

    # Save the model at the end of each epoch
    epoch_save_path = f"{model_save_path}-epoch-{epoch+1}"
    model.save(epoch_save_path)
    logging.info(f"Model saved after epoch {epoch+1} to {epoch_save_path}")
    
    # Evaluate the model using the validation set
    model.model.eval()  # Set model to evaluation mode
    eval_corr = evaluator(model)  # Only one value is returned by the evaluator
    logging.info(f"Epoch {epoch+1}: Correlation: {eval_corr}")


# Save the final model after all epochs
model.save(model_save_path + "-final")
logging.info(f"Training complete. Final model saved to {model_save_path + '-final'}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jhgan/ko-sroberta-multitask and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2024-10-09 15:36:16 - Use pytorch device: cuda
2024-10-09 15:36:17 - Starting training...


2024-10-09 15:39:08 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-1
2024-10-09 15:39:09 - Model saved after epoch 1 to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-1
2024-10-09 15:39:09 - CECorrelationEvaluator: Evaluating the model on aug-dev dataset:
2024-10-09 15:39:22 - Correlation:	Pearson: 0.8666	Spearman: 0.8231
2024-10-09 15:39:22 - Epoch 1: Correlation: 0.8230537371159695


2024-10-09 15:42:13 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-2
2024-10-09 15:42:14 - Model saved after epoch 2 to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-2
2024-10-09 15:42:14 - CECorrelationEvaluator: Evaluating the model on aug-dev dataset:
2024-10-09 15:42:27 - Correlation:	Pearson: 0.8912	Spearman: 0.8362
2024-10-09 15:42:27 - Epoch 2: Correlation: 0.8361504895354516


2024-10-09 15:45:19 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-3
2024-10-09 15:45:20 - Model saved after epoch 3 to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-3
2024-10-09 15:45:20 - CECorrelationEvaluator: Evaluating the model on aug-dev dataset:
2024-10-09 15:45:33 - Correlation:	Pearson: 0.9021	Spearman: 0.8399
2024-10-09 15:45:33 - Epoch 3: Correlation: 0.8398835531855577


...

2024-10-09 15:54:35 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-6
2024-10-09 15:54:35 - Model saved after epoch 6 to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-6
2024-10-09 15:54:35 - CECorrelationEvaluator: Evaluating the model on aug-dev dataset:
2024-10-09 15:54:48 - Correlation:	Pearson: 0.9155	Spearman: 0.8457
2024-10-09 15:54:48 - Epoch 6: Correlation: 0.8457410050085337


2024-10-09 15:57:40 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-7
2024-10-09 15:57:41 - Model saved after epoch 7 to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-7
2024-10-09 15:57:41 - CECorrelationEvaluator: Evaluating the model on aug-dev dataset:
2024-10-09 15:57:54 - Correlation:	Pearson: 0.9166	Spearman: 0.8425
2024-10-09 15:57:54 - Epoch 7: Correlation: 0.8425036993982152


2024-10-09 16:00:45 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-8
2024-10-09 16:00:46 - Model saved after epoch 8 to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-8
2024-10-09 16:00:46 - CECorrelationEvaluator: Evaluating the model on aug-dev dataset:
2024-10-09 16:00:59 - Correlation:	Pearson: 0.9163	Spearman: 0.8342
2024-10-09 16:00:59 - Epoch 8: Correlation: 0.8341547722933927


2024-10-09 16:03:50 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-9
2024-10-09 16:03:51 - Model saved after epoch 9 to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-9
2024-10-09 16:03:51 - CECorrelationEvaluator: Evaluating the model on aug-dev dataset:
2024-10-09 16:04:04 - Correlation:	Pearson: 0.9208	Spearman: 0.8460
2024-10-09 16:04:04 - Epoch 9: Correlation: 0.846029466621202


2024-10-09 16:06:55 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-10
2024-10-09 16:06:55 - Model saved after epoch 10 to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-epoch-10
2024-10-09 16:06:55 - CECorrelationEvaluator: Evaluating the model on aug-dev dataset:
2024-10-09 16:07:08 - Correlation:	Pearson: 0.9172	Spearman: 0.8438
2024-10-09 16:07:08 - Epoch 10: Correlation: 0.8438370429863699
2024-10-09 16:07:08 - Save model to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-final
2024-10-09 16:07:09 - Training complete. Final model saved to output/training_jhgan-ko-sroberta-multitask_binary_2024-10-09_15-36-16-final


In [ ]:
# aug_data_3.5turbo1106.jsonl: 1.0으로 PAIR: MAP 0.4955 MRR 0.4955
# aug_data_3.5turbo1106.jsonl: 1.0,-1.0 위 코드로 PAIR: MAP 0.7197 MRR 0.7242
# 